<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>

<h2 style="color: #FFA07A;">7. Aplicação do modelo XGBoost</h2>

In [1]:
#
import ipywidgets as widgets
import time

# --- Criar um aplicativo HTML para exibir o efeito de escrita ---
output = widgets.HTML(value="<div></div>")
display(output)

# --- Texto formatado para o efeito de escrita ---
texto = """
<div>
    <p>🔸 <span style="font-weight: bold;">O que faz o modelo?</span><br>
    O modelo estima a distribuição de <b>pessoas</b> ou <b>fluxos de mobilidade</b> com base na <b>acessibilidade</b> e na disponibilidade de serviços essenciais. Avalia a atratividade de diferentes áreas, tendo em conta fatores como <b>centros de saúde</b>, <b>farmácias</b>, <b>supermercados</b> e <b>parques ou jardins</b>, entre outros. Em áreas com maior concentração destes serviços, a presença de pessoas tende a ser superior; já em áreas com menor oferta, espera-se menor circulação. Esta abordagem permite perceber como a infraestrutura urbana influencia a movimentação e a distribuição da população.</p>
    
    <p>🔸 <span style="font-weight: bold;">O que são agentes? Como são distribuídos?</span><br>
    Os agentes representam <b>pessoas simuladas</b> no modelo. Os edifícios com maior atratividade recebem mais agentes, refletindo uma maior presença ou fluxo de pessoas nesses locais. Assim, é possível compreender como a oferta de serviços influencia a dinâmica urbana e os padrões de deslocação.</p>

    <p>🔸 <span style="font-weight: bold;">O que é a atratividade e qual a sua importância?</span><br>
    A <b>atratividade</b> mede o potencial de um edifício para atrair pessoas, com base na proximidade e diversidade de serviços essenciais. Quanto mais acessíveis e variados forem os serviços (como centros de saúde, farmácias, etc.), maior será a atratividade do local. O modelo utiliza este índice para distribuir os agentes de forma proporcional à atratividade de cada edifício.</p>
    
    <p>A atratividade utiliza os <b>pesos definidos na secção anterior</b>.</p>
    <u>Note-se que as variáveis <b>distância média aos serviços</b> e <b>população com 65 anos ou mais</b> são transformadas com o logaritmo para garantir uma distribuição mais equilibrada dos agentes.</u></p>
    
    <p>🔸 <span style="font-weight: bold;">Por que utilizar este modelo em vez do K-means?</span><br> 
    Ao contrário do <b>K-means</b>, que apenas agrupa áreas com base na proximidade espacial, o modelo <b>XGBoost</b> considera múltiplas variáveis, incluindo a <b>acessibilidade a serviços</b> e a <b>população residente</b>. Isto permite uma distribuição mais precisa e interpretável das pessoas, com base em fatores relevantes para o planeamento urbano.</p>
    
    <p>🔸 <span style="font-weight: bold;">O que é a análise SHAP?</span><br> 
    A <b>análise SHAP</b> (Shapley Additive Explanations) é uma técnica de <b>inteligência artificial explicável</b> que permite perceber como cada variável contribui para os resultados do modelo. Baseia-se na teoria dos jogos e fornece explicações justas e coerentes, considerando as interações entre todas as variáveis. É uma ferramenta poderosa para interpretar modelos complexos, ao contrário de métodos como o <b>LIME</b>, que oferecem explicações apenas locais (caso a caso).</p>
</div>
"""

# --- Efeito de escrita carácter a carácter (mantém o HTML) ---
texto_html = """
<div style="background-color: #FFFFFF; color: #333333; padding: 15px; 
            border-left: 5px solid #FFA500; font-family: Arial, sans-serif; 
            text-align: justify; font-size: 16px; line-height: 1.6;">
"""

for palavra in texto.split():
    texto_html += palavra + " "
    output.value = texto_html + "</div>"  
    time.sleep(0.10)  # Ajustar velocidade

# --- Garantir que o texto completo é exibido no final ---
output.value = texto_html + "</div>"

HTML(value='<div></div>')

In [2]:
from IPython.display import Javascript, display
# hide-me
display(Javascript('window.cellVisibilityManager.hideCells();'))

# --- Importar as bibliotecas ---
ipython = get_ipython()
ipython.run_line_magic("run", "1.preparacao_bibliotecas.ipynb")

# --- # Executar o notebook base ---
input_pkl_path = "df_servicos_clusters.pkl"
with open(input_pkl_path, 'rb') as pkl_file:
    data = pickle.load(pkl_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
# hide-me
display(Javascript('window.cellVisibilityManager.hideCells();'))

# --- Iniciar contagem de tempo ---
start_time = time.time()

# --- Funções auxiliares ---
def update_status(message):
    """Atualiza a mensagem no terminal."""
    sys.stdout.write("\r" + message)
    sys.stdout.flush()

def clear_status():
    """Limpa a mensagem do terminal."""
    sys.stdout.write("\r" + " " * 100 + "\r")
    sys.stdout.flush()

def print_moran_results(label, moran_obj):
    """Imprime os resultados do índice de Moran com um rótulo personalizado."""
    print(f"{label} Moran's I: {moran_obj.I}")
    print(f"p-value (simulação): {moran_obj.p_sim}")

def get_legenda_agentes():
    """Retorna o HTML da legenda para 'agentes'."""
    return """
    <div style="position: fixed;
                top: 10px; right: 10px; background: white; z-index:9999;
                font-size:14px; padding:5px; border-radius:5px;
                box-shadow: 0 0 5px rgba(0,0,0,0.5);">
        <strong>Agentes:</strong>
        <i style="background: gray; width: 15px; height: 15px; display: inline-block; border: 1px solid black;"></i> Ausência
        <i style="background: red; width: 15px; height: 15px; display: inline-block; border: 1px solid black;"></i> Presença
    </div>
    """

def get_legenda_classe_atratividade():
    """Retorna o HTML da legenda para a classe de atratividade."""
    return """
    <div style="position: fixed;
                top: 10px; right: 10px; background: white; z-index:9999;
                font-size:14px; padding:5px; border-radius:5px;
                box-shadow: 0 0 5px rgba(0,0,0,0.5);">
        <strong>Classe de atratividade:</strong><br>
        <i style="background: blue; width: 15px; height: 15px; display:inline-block; border:1px solid black;"></i> Baixa<br>
        <i style="background: orange; width: 15px; height: 15px; display:inline-block; border:1px solid black;"></i> Média<br>
        <i style="background: red; width: 15px; height: 15px; display:inline-block; border:1px solid black;"></i> Alta<br>
    </div>
    """

# --- Função para winsorização de outliers ---
def winsorize_series(s, lower_quantile=0.00, upper_quantile=0.95):
    """Limita os valores extremos de uma série com base nos quantis indicados."""
    lower = s.quantile(lower_quantile)
    upper = s.quantile(upper_quantile)
    return s.clip(lower, upper)

# Mensagem inicial
update_status("A calcular dados, aguarde... ")

# --- Carregar e preparar dataframe ---
# O dataframe está assumido em 'data' com a chave 'df'
df_servicos = data['df']

# Garantir que é um GeoDataFrame e definir CRS para EPSG:4326
df_servicos = gpd.GeoDataFrame(df_servicos, geometry='geometry').set_crs(epsg=4326)

# Reprojetar para um CRS projetado (ex: EPSG:3763) para cálculos mais precisos
df_servicos_proj = df_servicos.to_crs(epsg=3763)

# Calcular o centróide do conjunto de dados (usando união das geometrias) para visualização
centro_geom = df_servicos_proj.geometry.union_all().centroid
centro = (centro_geom.y, centro_geom.x)

# --- Calcular a variável "atratividade" ---
# A atratividade é calculada usando vários serviços com pesos específicos
df_servicos["atratividade"] = (
    df_servicos["Supermercados"].fillna(0) * 0.1614 +
    df_servicos["Bancos"].fillna(0) * 0.1522 +
    df_servicos["Farmacias"].fillna(0) * 0.1504 +
    df_servicos["CTT"].fillna(0) * 0.1499 +
    df_servicos["Parques e jardins"].fillna(0) * 0.1323 +
    df_servicos["Centro Saude"].fillna(0) * 0.1292 +
    df_servicos["Hospitais"].fillna(0) * 0.0876 +
    df_servicos["pop_64_mais"].fillna(0) * 0.0322 +
    df_servicos["distancia_media_servicos"].fillna(0) * 0.0049 
)

# Aplicar transformação logarítmica e winsorização para reduzir o efeito dos outliers
df_servicos["atratividade"] = np.log1p(df_servicos["atratividade"])
df_servicos["atratividade"] = winsorize_series(df_servicos["atratividade"], 0.0, 0.95)

# Medir o tempo de execução
end_time = time.time()
tempo_decorrido = end_time - start_time

# --- 4. Autocorrelação Espacial (Moran’s I) para ATRATIVIDADE ---
# Criar pesos espaciais usando KNN com 45 vizinhos
w = ps.weights.KNN.from_dataframe(df_servicos, k=45)
w.transform = 'r'
moran = esda.Moran(df_servicos["atratividade"], w)
print_moran_results("Atratividade", moran)

# --- 5. Transformar variáveis e criar novas colunas ---
# Transformação logarítmica da distância e população
df_servicos["distancia_media_servicos_log"] = np.log1p(df_servicos["distancia_media_servicos"])
df_servicos["pop_64_mais_log"] = np.log1p(df_servicos["pop_64_mais"])

# Adicionar coordenadas (x, y) dos centróides projectados
df_servicos['coord_x'] = df_servicos_proj.geometry.centroid.x
df_servicos['coord_y'] = df_servicos_proj.geometry.centroid.y

# --- 6. Preparar dados para modelação ---
variaveis = [
    "Centro Saude", "Farmacias", "Supermercados",
    "Parques e jardins", "Hospitais", 
    "CTT", "distancia_media_servicos_log", 
    "pop_64_mais_log", "coord_x", "coord_y"
]
X = df_servicos[variaveis].fillna(0)

# Inicializar a coluna 'agentes' se não existir, com base na atratividade
if 'agentes' not in df_servicos.columns:
    num_agentes_default = 22000
    soma_atratividade = df_servicos["atratividade"].sum()
    df_servicos["proporcao"] = df_servicos["atratividade"] / soma_atratividade
    df_servicos["agentes"] = (df_servicos["proporcao"] * num_agentes_default).round()

y = df_servicos["agentes"]

# Dividir dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 7. Modelação com XGBoost Regressor (Normalização de agentes incluída) ---
# (A definição da classe XGBRegressorNormalized mantém-se inalterada...)

class XGBRegressorNormalized:
    def __init__(self, total_agents=22000, atratividade_minima=2.6,
                 n_estimators=100, max_depth=3, learning_rate=0.3, random_state=42):
        self.total_agents = total_agents
        self.atratividade_minima = atratividade_minima
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.random_state = random_state
        self.model = XGBRegressor(n_estimators=self.n_estimators,
                                  max_depth=self.max_depth,
                                  learning_rate=self.learning_rate,
                                  random_state=self.random_state,
                                  reg_alpha=1.0,
                                  reg_lambda=1.0)

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X, df_meta=None):
        if df_meta is None:
            raise ValueError("É necessário passar df_meta com a coluna 'atratividade'.")

        preds = self.model.predict(X)
        preds = np.clip(preds, 0, None)

        # Pré-atribuir 1 agente a locais com atratividade ≥ limiar
        agentes = np.zeros(len(preds), dtype=int)
        mascara_obrigatorios = df_meta["atratividade"].values >= self.atratividade_minima
        agentes[mascara_obrigatorios] = 1
        restantes = self.total_agents - agentes.sum()

        # Anular previsões para locais já atribuídos
        preds[mascara_obrigatorios] = 0

        # Normalizar as previsões restantes
        soma_prevista_restante = preds.sum()
        fator_normalizacao = restantes / soma_prevista_restante if soma_prevista_restante > 0 else 0
        scaled_predictions = preds * fator_normalizacao
        floored = np.floor(scaled_predictions)
        residuos = scaled_predictions - floored

        agentes += floored.astype(int)
        soma_atual = agentes.sum()
        diferenca = self.total_agents - soma_atual

        # Ajuste final de arredondamento
        if diferenca > 0:
            indices = np.argsort(-residuos)[:diferenca]
            for idx in indices:
                agentes[idx] += 1
        elif diferenca < 0:
            indices = np.argsort(residuos)[:abs(diferenca)]
            for idx in indices:
                agentes[idx] -= 1

        return agentes

    def get_params(self, deep=True):
        return {
            "total_agents": self.total_agents,
            "atratividade_minima": self.atratividade_minima,
            "n_estimators": self.n_estimators,
            "max_depth": self.max_depth,
            "learning_rate": self.learning_rate,
            "random_state": self.random_state
        }

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        self.model = XGBRegressor(n_estimators=self.n_estimators,
                                  max_depth=self.max_depth,
                                  learning_rate=self.learning_rate,
                                  random_state=self.random_state)
        return self

# Calcular o Moran's I para agentes
moran_agentes = esda.Moran(df_servicos["agentes"], w)
print_moran_results("Agentes", moran_agentes)

# Treinar o modelo regressivo customizado e obter previsões normalizadas
modelo_xgb = XGBRegressorNormalized(total_agents=22000, n_estimators=100, max_depth=3, learning_rate=0.3, random_state=42)
modelo_xgb.fit(X_train, y_train)
df_servicos["agentes"] = modelo_xgb.predict(X, df_meta=df_servicos)
print("Soma total dos agentes após normalização:", df_servicos["agentes"].sum())

# --- 8. Cálculo dos resíduos e avaliação da autocorrelação espacial ---
df_servicos["residuos"] = y - modelo_xgb.model.predict(X)
moran_residuos = esda.Moran(df_servicos["residuos"], w)
print_moran_results("Resíduos (KNN)", moran_residuos)

# Usar o método DistanceBand como abordagem alternativa para pesos espaciais
threshold_distance = 500
w_distance = DistanceBand.from_dataframe(df_servicos_proj, threshold=threshold_distance, silence_warnings=True)
w_distance.transform = 'r'
moran_residuos_distance = esda.Moran(df_servicos["residuos"], w_distance)
print_moran_results("Resíduos (DistanceBand)", moran_residuos_distance)

# --- 9. Análise SHAP para o modelo de regressão ---
explainer = shap.TreeExplainer(modelo_xgb.model)
shap_values = explainer.shap_values(X)

# Dicionário de variáveis
label_map = {
    'numero_servicos_proximos': 'Número total de serviços próximos',
    'pop_64_mais_log': 'População com 65 anos ou + (log)',
    'distancia_media_servicos_log': 'Distância média aos serviços (log)',
    'Centro Saude': 'Centro de saúde',
    'Farmacias': 'Farmácias',
    'Hospitais': 'Hospitais',
    'Supermercados': 'Supermercados',
    'Bancos': 'Bancos',
    'Parques e jardins': 'Parques ou jardins',
    'CTT': 'CTT',
    'coord_x': 'Coordenada X',
    'coord_y': 'Coordenada Y'
}

# ------- aliases para o mapa vindos do label_map -------
alias_map = {
    **label_map,
    'atratividade': 'Atratividade',
    'atratividade_formatada': 'Atratividade',
    'distancia_media_servicos_formatada': 'Distância média',
    'numero_servicos_proximos': 'Número total de serviços próximos',
    'pop_64_mais': 'População 65+',
    'classe_atratividade_num': 'Classe de atratividade',
    'agentes': 'Agentes',
}
def mk_aliases(campos):
    return [f"{alias_map.get(c, c)}:" for c in campos]
# -------------------------------------------------------

def gerar_summary_plot_base64():
    """Gera o gráfico sumário SHAP (beeswarm) mostrando as 10 principais variáveis, com rótulos em português."""
    import matplotlib.pyplot as plt
    X_shap = X.rename(columns=label_map)
    plt.figure(figsize=(10, 6))
    shap.summary_plot(
        shap_values,
        X_shap,
        plot_type="dot",
        show=False,
        max_display=10
    )
    fig = plt.gcf()
    ax = plt.gca()

    # Texto explicativo personalizado acima do gráfico
    ax.text(
        0.5, 1.02,
        "Pontos à esquerda → diminuem a previsão; à direita → aumentam a previsão.",
        ha="center", va="bottom", transform=ax.transAxes,
        fontsize=10, style="italic"
    )
    ax.grid(True, linestyle="--", linewidth=0.5, alpha=0.7)

    # Alterar o rótulo do eixo X para português
    ax.set_xlabel("Valor SHAP (impacto na previsão)", fontsize=12)

    # Alterar todos os rótulos de texto soltos (ex: "Feature value")
    for txt in fig.findobj(match=plt.Text):
        if txt.get_text() == "Feature value":
            txt.set_text("Valor da variável")

 # --- Alterar "Low" e "High" directamente na barra de cores ---
    # Procurar a colorbar entre todos os eixos da figura
    for axes in fig.axes:
        if hasattr(axes, 'get_ylabel') and axes.get_ylabel() == "Feature value":
             # Combinar y e x tick labels da colorbar
            colorbar_ticklabels = axes.get_yticklabels() + axes.get_xticklabels()
            for label in colorbar_ticklabels:
                if label.get_text() == "Low":
                    label.set_text("Baixo")
                elif label.get_text() == "High":
                    label.set_text("Alto")
            # Forçar actualização dos rótulos
            axes.figure.canvas.draw_idle()

    buf = io.BytesIO()
    plt.tight_layout()
    plt.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    plt.close()
    return base64.b64encode(buf.read()).decode()
    
def gerar_shap_bar_plot_base64():
    """Gera um bar plot com as importâncias médias absolutas de SHAP (com nomes amigáveis)."""
    import matplotlib.pyplot as plt
    X_shap = X.rename(columns=label_map)
    plt.figure(figsize=(8, 5))
    shap.summary_plot(
        shap_values,
        X_shap,
        plot_type="bar",
        show=False,
        max_display=10    
    )
    ax = plt.gca()
    ax.set_title("Importância Médias Absolutas de SHAP", fontsize=14, pad=15)
    ax.set_xlabel("Média |valor SHAP|", fontsize=12)
    buf = io.BytesIO()
    plt.tight_layout()
    plt.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    plt.close()
    return base64.b64encode(buf.read()).decode()

# Criar colunas formatadas para exibição dos valores
df_servicos["atratividade_formatada"] = df_servicos["atratividade"].apply(lambda x: f"{x:.1f}")
df_servicos["distancia_media_servicos_formatada"] = df_servicos["distancia_media_servicos"].apply(lambda x: f"{x:.1f}")

# --- 10. Classificação da atratividade em 3 classes usando Natural Breaks ---
nb = mapclassify.NaturalBreaks(df_servicos["atratividade"], k=3)
df_servicos["classe_atratividade_num"] = nb.yb
print("Limites das classes (Natural Breaks):", nb.bins)
labels = {0: "Baixa", 1: "Média", 2: "Alta"}
df_servicos["classe_atratividade"] = df_servicos["classe_atratividade_num"].map(labels)

X_class = X.copy()
y_class = df_servicos["classe_atratividade_num"]

X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

# Treinar o modelo de classificação
modelo_xgb_clf = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)
modelo_xgb_clf.fit(X_train_clf, y_train_clf)
predicoes_clf = modelo_xgb_clf.predict(X_test_clf)

accuracy_clf = accuracy_score(y_test_clf, predicoes_clf)

# --- 11. Criação do dashboard com Dash, mapa interativo e gráfico SHAP ---
app = dash.Dash(__name__)

app.layout = html.Div(
    [  
        html.H1(
            "Distribuição de agentes e atratividade",
            style={
                'textAlign': 'center',
                'color': 'white',
                'backgroundColor': '#000',
                'padding': '10px',
                'margin': '0'
            }
        ),
        html.Div(
            [
                html.Label(
                    "Selecione o atributo para visualizar no mapa:",
                    style={
                        'color': 'white',
                        'textAlign': 'center',
                        'display': 'block'
                    }
                ),
                dcc.Dropdown(
                    id='atributo-dropdown',
                    options=[
                        {'label': 'Agentes', 'value': 'agentes'},
                        {'label': 'Atratividade', 'value': 'atratividade'},
                        {'label': 'Classes de atratividade', 'value': 'classe_atratividade_num'}
                    ],
                    value='agentes',
                    style={
                        'width': '50%',
                        'margin': 'auto',
                        'textAlign': 'center',
                        'textAlignLast': 'center'
                    }
                )
            ],
            style={'textAlign': 'center', 'margin-bottom': '10px', 'backgroundColor': '#000'}
        ),
        html.Div(
            [
                html.Iframe(
                    id='mapa-interativo',
                    style={'width': '100%', 'height': '600px', 'border': 'none'}
                )
            ]
        ),
        html.Div(
            [
                html.Label(
                    "Gráfico de Resumo SHAP:",
                    style={
                        'color': 'white',
                        'textAlign': 'center',
                        'margin': '0',
                        'padding': '10px'
                    }
                ),
                # Beeswarm 
                html.Img(
                    src='data:image/png;base64,{}'.format(gerar_summary_plot_base64()),
                    style={'width': '100%', 'margin': '0', 'padding': '0'}
                ),
                html.Img(
                    src='data:image/png;base64,{}'.format(gerar_shap_bar_plot_base64()),
                    style={'width': '100%', 'margin': '0', 'padding': '0', 'marginTop': '20px'}
                )
            ],
            style={
                'backgroundColor': '#000',
                'textAlign': 'center',
                'margin-top': '20px'
            }
        )
    ],
    style={'backgroundColor': '#000'}
)

@app.callback(
    Output('mapa-interativo', 'srcDoc'),
    Input('atributo-dropdown', 'value')
)
def atualizar_mapa(atributo):
    return criar_mapa_interativo(df_servicos, atributo)
    
def criar_mapa_interativo(df, atributo):
    """
    Função para criar um mapa interativo com base no atributo selecionado.
    Usa CartoDB Dark Matter para 'atratividade' e 'classes de atratividade',
    e CartoDB Positron para os outros atributos.
    """
    # Escolher tiles base consoante o atributo
    if atributo in ['atratividade', 'classe_atratividade_num']:  # 'classe_atratividade_num' usa o mesmo estilo que 'atratividade'
        tiles_style = 'CartoDB dark_matter'  # Fundo escuro para 'atratividade' e 'classes de atratividade'
    else:
        tiles_style = 'CartoDB positron' # Fundo claro para 'agentes' e outros atributos
    
     # Criar objecto mapa com o estilo seleccionado
    mapa = folium.Map(
        location=[41.1500, -8.6291],
        zoom_start=13,
        min_zoom=13,
        tiles=tiles_style,
        control_scale=False
    )

    if atributo == 'agentes':
        # Legenda personalizada para agentes
        legenda_html = get_legenda_agentes()
        mapa.get_root().html.add_child(folium.Element(legenda_html))
        fields = ["agentes", "atratividade", "numero_servicos_proximos"]
        folium.GeoJson(
            data=df,
            name="Clusters",
            style_function=lambda feature: {
                "fillColor": "red" if feature["properties"][atributo] else "gray",
                "color": "black",
                "weight": 0.5,
                "fillOpacity": 0.6,
            },
            tooltip=folium.GeoJsonTooltip(
                fields=fields,
                aliases=mk_aliases(fields),
            ),
        ).add_to(mapa)

    elif atributo == 'classe_atratividade_num':
        # GeoJson + legenda para as classes
        colors = {0: "blue", 1: "orange", 2: "red"}
        fields = ["agentes", "atratividade_formatada", "classe_atratividade_num"]
        folium.GeoJson(
            data=df,
            name="Clusters",
            style_function=lambda feature: {
                "fillColor": colors.get(feature["properties"][atributo], "gray"),
                "color": "black",
                "weight": 0.5,
                "fillOpacity": 0.6,
            },
            tooltip=folium.GeoJsonTooltip(
                fields=fields,
                aliases=mk_aliases(fields),
            ),
        ).add_to(mapa)
        legenda_html = get_legenda_classe_atratividade()
        mapa.get_root().html.add_child(folium.Element(legenda_html))

    else:
         # Variáveis contínuas: atratividade ou outras
        colormap = linear.YlOrRd_09.scale(df[atributo].min(), df[atributo].max())
        # Legenda padrão para atributos não atratividade
        if atributo != 'atratividade':
            colormap.caption = f"Escala de {alias_map.get(atributo, atributo)}"
            colormap.add_to(mapa)
        
        # Desenhar GeoJson
        fields = ["agentes", "atratividade_formatada", "Centro Saude", "Farmacias", "Supermercados", 
                  "Parques e jardins", "distancia_media_servicos_formatada", "pop_64_mais", "numero_servicos_proximos"]
        folium.GeoJson(
            data=df,
            name="Clusters",
            style_function=lambda feature: {
                "fillColor": colormap(feature["properties"][atributo]),
                "color": "black",
                "weight": 0.5,
                "fillOpacity": 0.6,
            },
            tooltip=folium.GeoJsonTooltip(
                fields=fields,
                aliases=mk_aliases(fields)
            ),
        ).add_to(mapa)

        # Legenda personalizada se o atributo for 'atratividade'
        if atributo == 'atratividade':
            min_val = df[atributo].min()
            max_val = df[atributo].max()
            gradient = 'linear-gradient(to right, #ffffb2, #bd0026)'
            legend_html = f'''
            <div style="position: fixed; top: 10px; right: 10px; background: white; z-index:9999;
                         padding:10px; border-radius:4px; box-shadow:0 0 5px rgba(0,0,0,0.5); font-size:12px; color:black;">
                <div style="font-weight:bold; margin-bottom:4px;">Atratividade</div>
                <div style="width:200px; height:10px; background: {gradient}; margin:4px 0;"></div>
                <div style="display:flex; justify-content:space-between;">
                  <span>{min_val:.1f}</span><span>{max_val:.1f}</span>
                </div>
            </div>
            '''
            mapa.get_root().html.add_child(folium.Element(legend_html))

    return mapa._repr_html_()

# --- 12. Calcular métricas do modelo de regressão ---
def calcular_metricas(y_test, y_pred):
    """
    Calcula e retorna várias métricas de avaliação para o modelo de regressão.
    """
    epsilon = 1e-12  # Para evitar divisão por zero
    y_test_safe = np.where(np.abs(y_test) < epsilon, epsilon, y_test)
    rmsle = np.sqrt(mean_squared_log_error(np.maximum(y_test, epsilon), np.maximum(y_pred, epsilon)))
    medape = np.median(np.abs((y_test - y_pred) / y_test_safe)) * 100
    bias = np.mean(y_pred - y_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    cvrmse = (rmse / np.mean(y_test_safe)) * 100
    return {
        "R²": r2_score(y_test, y_pred),
        "RMSE": rmse,
        "MAE": mean_absolute_error(y_test, y_pred),
        "RMSLE": rmsle,
        "MedAPE (%)": medape,
        "Bias": bias,
        "CVRMSE (%)": cvrmse
    }

# Avaliar o modelo de regressão com o conjunto de teste
y_pred_test = modelo_xgb.model.predict(X_test)
metrics_rf = calcular_metricas(y_test, y_pred_test)

# --- GUARDAR O DATAFRAME ---
df_servicos.to_pickle("df_servicos_atratividade.pkl")

# Limpar mensagem de estado
clear_status()

print("Dados calculados com sucesso, exibindo tabela, painel e gráficos...\n")
print("### Métricas do Modelo XGBoost (Regressão) ###")
for k, v in metrics_rf.items():
    print(f"{k}: {v:.4f}")

# --- 13. Função para encontrar uma porta livre ---
def find_free_port():
    """Encontra uma porta livre entre 8000 e 9000."""
    while True:
        port = random.randint(8000, 9000)
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(("localhost", port)) != 0:
                return port

# Atribuir uma porta disponível e segura
port = find_free_port()

# --- Lançar a aplicação interativa quando o ficheiro for executado ---
if __name__ == "__main__":
    app.run(debug=False, port=port)

# ---- Mensagem final---
print("\033[92m[INFO] Após análise, pode continuar.\033[0m")

<IPython.core.display.Javascript object>

Atratividade Moran's I: 0.9573731132238062
p-value (simulação): 0.001
Agentes Moran's I: 0.7343335038876384
p-value (simulação): 0.001
Soma total dos agentes após normalização: 22000
Resíduos (KNN) Moran's I: 0.021131541386164587
p-value (simulação): 0.001
Resíduos (DistanceBand) Moran's I: 0.0007963443834120592
p-value (simulação): 0.006
Limites das classes (Natural Breaks): [1.94140049 2.35929904 2.68629528]
Dados calculados com sucesso, exibindo tabela, painel e gráficos...                                 

### Métricas do Modelo XGBoost (Regressão) ###
R²: 0.9425
RMSE: 0.0342
MAE: 0.0055
RMSLE: 0.0242
MedAPE (%): 0.0566
Bias: -0.0002
CVRMSE (%): 3.4877


[INFO] Após análise, pode continuar.


In [5]:
#
import ipywidgets as widgets
import time
# --- Criar um aplicativo HTML para exibir o efeito de escrita ---
output = widgets.HTML(value="<div></div>")
display(output)

# Texto formatado corretamente para efeito de digitação
texto = """
<div> 
    <p style="text-align: justify;"> 
        <b>Explore os mapas e os gráficos SHAP e, em seguida, responda às questões.</b>
    </p>
</div>
"""

# Criar efeito de digitação dentro do HTML mantendo a formatação original
texto_html = """
<div style="background-color: #FFFFFF; color: #333333; padding: 15px; 
            border-left: 5px solid #6A0DAD; font-family: Arial, sans-serif; 
            text-align: justify; font-size: 16px; line-height: 1.6;">
"""
for palavra in texto.split():
    texto_html += palavra + " "
    output.value = texto_html + "</div>"  
    time.sleep(0.10)  # Ajustar velocidade

# --- Garantir que o texto completo é exibido no final ---
output.value = texto_html + "</div>" 

HTML(value='<div></div>')

<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>

In [3]:
import os
import pandas as pd
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Pasta específica para guardar as respostas ---
pasta_respostas = "./respostas"
os.makedirs(pasta_respostas, exist_ok=True)

# --- Questão de escolha múltipla (Q6) ---
pergunta1 = widgets.HTML(
    value="<b style='font-size: 16px;'>6. Qual é a variável que mais influencia o modelo, seja com impacto positivo ou negativo?</b>"
)
opcoes_pergunta1 = widgets.RadioButtons(
    options=[
        "a) População com 65 anos ou + (log).",
        "b) Farmácia.",
        "c) Distância média aos serviços (log).",
    ],
    description='',
    style={'description_width': 'initial'},
    value=None
)
opcoes_pergunta1.layout = widgets.Layout(width='80%')

# --- Q7 ---
pergunta2 = widgets.HTML(
    value=(
        "<b style='font-size: 16px;'>7. A afirmação é verdadeira ou falsa? </b><br>"
        "<i>\"Na área oriental do município do Porto (freguesia de Campanhã), "
        "o modelo atribui valores baixos de atratividade, devido à boa oferta de serviços "
        "disponíveis para a população com 65 anos ou mais.\"</i>"
    )
)
opcoes_pergunta2 = widgets.RadioButtons(
    options=[
        "a) Verdadeira – há boa oferta de serviços.",
        "b) Falsa – há pouca oferta de serviços.",
        "c) Falsa – o modelo não considera serviços."
    ],
    description='',
    style={'description_width': 'initial'},
    value=None
)
opcoes_pergunta2.layout = widgets.Layout(width='80%')

# --- Q8 ---
pergunta3 = widgets.HTML(
    value=(
        "<b style='font-size: 16px;'>8. A afirmação é verdadeira ou falsa? </b><br>"
        "<i>\"A atratividade de um edifício, segundo o modelo XGBoost, depende exclusivamente "
        "do número de residentes com 65 anos ou mais, desconsiderando os serviços.\"</i>"
    )
)
opcoes_pergunta3 = widgets.RadioButtons(
    options=[
        "a) Verdadeira – o modelo considera apenas uma variável.",
        "b) Falsa – o modelo baseia-se apenas na localização.",
        "c) Falsa – o modelo também considera os serviços."
    ],
    description='',
    style={'description_width': 'initial'},
    value=None
)
opcoes_pergunta3.layout = widgets.Layout(width='80%')

# Botão e output
botao_gravar = widgets.Button(description='Gravar resposta', button_style='info')
output = widgets.Output()

def gravar_resposta(b):
    with output:
        clear_output()
        resposta1 = opcoes_pergunta1.value
        resposta2 = opcoes_pergunta2.value
        resposta3 = opcoes_pergunta3.value

        if not resposta1 or not resposta2 or not resposta3:
            print("Por favor, responda a todas as questões antes de gravar.")
            return

        dados = {
            "Resposta 1": [resposta1],
            "Resposta 2": [resposta2],
            "Resposta 3": [resposta3],
            "Data": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
        }

        ficheiro = os.path.join(pasta_respostas, "respostas_shap.xlsx")
        df = pd.DataFrame(dados)

        try:
            if not os.path.isfile(ficheiro):
                df.to_excel(ficheiro, index=False, engine='openpyxl')
                print(f"Ficheiro criado com sucesso em: {os.path.abspath(ficheiro)}")
            else:
                df_existente = pd.read_excel(ficheiro, engine='openpyxl')
                df_final = pd.concat([df_existente, df], ignore_index=True)
                df_final.to_excel(ficheiro, index=False, engine='openpyxl')
                print(f"Resposta adicionada ao ficheiro: {os.path.abspath(ficheiro)}")
            print("Resposta gravada com sucesso! Obrigado.")
            opcoes_pergunta1.value = None
            opcoes_pergunta2.value = None
            opcoes_pergunta3.value = None
        except Exception as e:
            print("Ocorreu um erro ao gravar o ficheiro. Detalhes:", e)
            print("Sugestões: verifique se o ficheiro não está aberto e se o 'openpyxl' está instalado.")

botao_gravar.on_click(gravar_resposta)

# Instruções
display(widgets.HTML("""
<b style="font-size: 18px;">Por favor, leia a explicação e responda às questões.</b><br><br>
O gráfico SHAP mostra o impacto de cada variável nas previsões do modelo. As variáveis estão listadas à esquerda.
A cor de cada ponto representa o valor da variável (azul = valor baixo, vermelho = valor alto), enquanto a posição horizontal mostra se o valor da variável aumentou ou diminuiu a previsão. Quanto mais distante do zero, maior o impacto da variável.
"""))

# Mostrar perguntas
display(pergunta1, opcoes_pergunta1)
display(pergunta2, opcoes_pergunta2)
display(pergunta3, opcoes_pergunta3)
display(botao_gravar, output)

<IPython.core.display.Javascript object>

HTML(value='\n<b style="font-size: 18px;">Por favor, leia a explicação e responda às quesões.</b><br><br>\n\nO…

RadioButtons(layout=Layout(width='80%'), options=('a) População com 65 anos ou + (log).', 'b) Farmácia.', 'c) …

HTML(value='\n<b style="font-size: 16px;">7. Classifique a afirmação: </b><br>\n<i>"Na área oriental do municí…

RadioButtons(layout=Layout(width='80%'), options=('a) Verdadeira – há boa oferta de serviços.', 'b) Falsa – há…

HTML(value='\n<b style="font-size: 16px;">8. Classifique a afirmação: </b><br>\n<i>"A atratividade de um edifí…

RadioButtons(layout=Layout(width='80%'), options=('a) Verdadeira – o modelo considera apenas uma variável.', '…

Button(button_style='info', description='Gravar resposta', style=ButtonStyle())

Output()

<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>

In [4]:
from IPython.display import display, HTML

# HTML containing the title
html_content = '<h2 style="color: #FFA07A;">7.1. Estatísticas do modelo XGBoost</h2>'

# Display the HTML content# Exibir o conteúdo HTML
display(HTML(html_content))

In [5]:
#
import ipywidgets as widgets
import time
# --- Criar um aplicativo HTML para exibir o efeito de escrita ---
output = widgets.HTML(value="<div></div>")
display(output)

# --- Texto formatado para o efeito de escrita ---
texto = """
<div>
    <p> 🔸 Para finalizar a nossa análise, vamos explorar estatísticas adicionais. Após esse processo, responda às questões.</p>
</div>
"""

# --- Efeito de escrita carácter a carácter (mantém o HTML) ---
texto_html = """
<div style="background-color: #FFFFFF; color: #333333; padding: 15px; 
            border-left: 5px solid #FFA500; font-family: Arial, sans-serif; 
            text-align: justify; font-size: 16px; line-height: 1.6;">
"""
for palavra in texto.split():
    texto_html += palavra + " "
    output.value = texto_html + "</div>"  
    time.sleep(0.10)   # Ajustar velocidade 

# --- Garantir que o texto completo é exibido no final ---
output.value = texto_html + "</div>" 

HTML(value='<div></div>')

In [5]:
# hide-me
display(Javascript('window.cellVisibilityManager.hideCells();'))

from matplotlib.ticker import MaxNLocator  
from io import StringIO
# --- Mapeamento para nomes legíveis das variáveis ---
label_map = {
    'numero_servicos_proximos': 'Número total de serviços próximos',
    'pop_64_mais_log': 'População com 65 anos ou + (log)',
    'distancia_media_servicos_log': 'Distância média aos serviços (log)',
    'Centro Saude': 'Centro de saúde',
    'Farmacias': 'Farmácias',
    'Hospitais': 'Hospitais',
    'Supermercados': 'Supermercados',
    'Bancos': 'Bancos',
    'Parques e jardins': 'Parques ou jardins',
    'CTT': 'CTT',
    'coord_x': 'Coordenada X',
    'coord_y': 'Coordenada Y'
}

# --- Carregar o ficheiro PKL (apenas uma vez) e armazenar o DataFrame globalmente ---
df_servicos = pd.read_pickle("df_servicos_atratividade.pkl")

colunas_numericas = df_servicos.select_dtypes(include=[np.number]).columns
df_servicos[colunas_numericas] = df_servicos[colunas_numericas].apply(
    lambda col: col.map(lambda x: 0 if abs(x) < 1e-10 else x)
)

# --- Criar classes de atratividade ---
_df = df_servicos.copy()
_df['classe_atratividade'] = pd.qcut(_df['atratividade'], q=3,
                                     labels=['Baixa', 'Média', 'Alta'])
_df['classe_atratividade_num'] = _df['classe_atratividade'].map(
    {'Baixa': 0, 'Média': 1, 'Alta': 2}
)
df_servicos = _df

# --- Funções auxiliares (fig_to_base64, generate_rich_tables_html, ...) ---
def fig_to_base64(fig):
    buf = BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight", facecolor="black")
    buf.seek(0)
    encoded = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    plt.close(fig)
    return f"data:image/png;base64,{encoded}"

# Função para gerar a matriz de correlação com os rótulos correctos
def render_correlation_matrix(correlacao):
    fig = px.imshow(
        correlacao, text_auto=True, color_continuous_scale='RdBu',
        template="plotly_dark"
    )
    fig.update_layout(
        paper_bgcolor='black', plot_bgcolor='black', height=800,
        xaxis=dict(tickangle=90),
        margin=dict(l=150, r=150, t=50, b=150),
    )
    return fig

def generate_rich_tables_html():
    console = Console(record=True, force_terminal=True, file=StringIO())
    # Tabela ANOVA 
    table_anova = Table(show_header=True, header_style="bold magenta",
                        title="Tabela Completa - ANOVA", style="white")
    for col in df_anova_final.columns:
        table_anova.add_column(col, justify="center")
    for _, row in df_anova_final.iterrows():
        row['Variável'] = label_map.get(row['Variável'], row['Variável'])
        table_anova.add_row(*[str(v) for v in row])

    # Tabela Tukey
    table_tukey = Table(show_header=True, header_style="bold green",
                        title="Tabela Completa - Teste Post-hoc de Tukey", style="white")
    for col in df_tukey_final.columns:
        table_tukey.add_column(col, justify="center")
    for _, row in df_tukey_final.iterrows():
        row['Variável'] = label_map.get(row['Variável'], row['Variável'])
        table_tukey.add_row(*[str(v) for v in row])

    console.print(table_anova)
    console.print(table_tukey)
    html_out = console.export_html(inline_styles=True)
    return (
        "<div style='display:flex;justify-content:center;'>"
        "<div style='text-align:center;margin:20px auto;max-width:90%;'>"
        f"{html_out}"
        "</div></div>"
    )

# --- Cálculo dos testes ANOVA e Tukey ---
def calcular_testes_reais(df):
    variaveis = [
        "Centro Saude", "Farmacias", "Supermercados", "Parques e jardins",
        "Hospitais", "CTT", "distancia_media_servicos_log", "pop_64_mais_log"
    ]
    lista_anova, lista_tukey = [], []
    for var in variaveis:
        modelo = smf.ols(f'Q("{var}") ~ C(classe_atratividade)', data=df).fit()
        anova_res = sm.stats.anova_lm(modelo, typ=2)

        if "C(classe_atratividade)" in anova_res.index:
            row = anova_res.loc["C(classe_atratividade)"]
            lista_anova.append({
                "Fonte": "C(classe_atratividade)", "Variável": label_map.get(var, var),
                "sum_sq": row["sum_sq"], "df": row["df"],
                "F": row["F"], "PR(>F)": row.get("PR(>F)", np.nan)
            })

        tukey = pairwise_tukeyhsd(
            endog=df[var], groups=df['classe_atratividade'], alpha=0.05
        )
        tukey_df = pd.DataFrame(
            tukey._results_table.data[1:], columns=tukey._results_table.data[0]
        )
        tukey_df['Variável'] = label_map.get(var, var)
        lista_tukey.append(tukey_df)

    return pd.DataFrame(lista_anova), pd.concat(lista_tukey, ignore_index=True)


# Executar os testes
_df_an, _df_tu = calcular_testes_reais(df_servicos)
df_anova_final, df_tukey_final = _df_an, _df_tu

# --- Preparar resumos (df_anova_resumo, df_tukey_resumo) ---
df_anova_resumo = (
    df_anova_final[df_anova_final['Fonte'] == 'C(classe_atratividade)']
    .loc[:, ['Variável', 'sum_sq', 'df', 'F', 'PR(>F)']]
    .rename(columns={
        'sum_sq': 'Soma dos Quadrados',
        'df': 'Graus de Liberdade',
        'F': 'Valor F',
        'PR(>F)': 'p-valor'
    }).sort_values('Variável')
)
df_anova_resumo['Soma dos Quadrados'] = df_anova_resumo['Soma dos Quadrados'].map(
    lambda x: f"{x:.2f}"
)
df_anova_resumo['Valor F'] = df_anova_resumo['Valor F'].map(
    lambda x: f"{x:.2f}"
)

df_tukey_resumo = (
    df_tukey_final[['Variável', 'group1', 'group2', 'meandiff', 'p-adj', 'reject']]
    .sort_values(['Variável', 'group1', 'group2'])
)
df_tukey_resumo['meandiff'] = df_tukey_resumo['meandiff'].map(
    lambda x: f"{x:.2f}"
)

# --- Função para criar a aplicação Dash 2 ---
def create_app():
    app = Dash(
        __name__,
        external_stylesheets=["https://codepen.io/chriddyp/pen/bWLwgP.css"]
    )

    app.layout = html.Div([
        html.H1(
            "Análise do modelo XGBoost",
            style={
                'text-align': 'center', 'color': 'white', 'background-color': '#000',
                'padding': '10px', 'font-weight': 'bold', 'font-size': '22px'
            }
        ),

        # TABS 
        dcc.Tabs(
            id="tabs", value="tab1",
            children=[
                dcc.Tab(
                    label="Matriz de Correlação", value="tab1",
                    style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px'},
                    selected_style={'backgroundColor': '#000', 'color': 'white',
                                    'padding': '10px', 'borderTop': '4px solid #ffcc00'}
                ),
                dcc.Tab(
                    label="Gráficos por variável e classe", value="tab2",
                    style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px'},
                    selected_style={'backgroundColor': '#000', 'color': 'white',
                                    'padding': '10px', 'borderTop': '4px solid #ffcc00'}
                ),
                dcc.Tab(
                    label="Teste ANOVA e Tukey", value="tab3",
                    style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px'},
                    selected_style={'backgroundColor': '#000', 'color': 'white',
                                    'padding': '10px', 'borderTop': '4px solid #ffcc00'}
                )
            ],
            style={'backgroundColor': '#000', 'border': '1px solid #333'}
        ),

        html.Div(id="content",
                 style={'backgroundColor': '#000', 'color': 'white', 'padding': '20px'})
    ], style={'backgroundColor': '#000', 'color': 'white', 'minHeight': '100vh',
              'margin': '0', 'padding': '0', 'overflow': 'hidden'})

    # CALLBACKS 
    @app.callback(Output("content", "children"), Input("tabs", "value"))
    def render_tab_content(tab):
        if tab == "tab1":
            features = [
                "Centro Saude", "Farmacias", "Supermercados", "Parques e jardins",
                "Hospitais", "CTT", "distancia_media_servicos_log",
                "pop_64_mais_log", "coord_x", "coord_y"
            ]
            features_with_labels = [label_map.get(f, f) for f in features]
            
            df_servicos_renamed = df_servicos.copy()
            df_servicos_renamed.columns = [label_map.get(col, col) for col in df_servicos.columns]
            
            correlacao = df_servicos_renamed[features_with_labels].corr()
            fig = render_correlation_matrix(correlacao)

            return html.Div([
                dcc.Graph(figure=fig, style={"marginBottom": "50px"})
            ])

        elif tab == "tab2":
            variaveis_para_plotar = [
                "Centro Saude", "Farmacias", "Supermercados",
                "Parques e jardins", "Hospitais", "CTT",
                "distancia_media_servicos", "pop_64_mais"
            ]
            palette_custom = {"Baixa": "cyan", "Média": "blue", "Alta": "orange"}
            imagens = []
            for var in variaveis_para_plotar:
                title = label_map.get(var, var)  # Pega o nome correto da variável
                fig, ax = plt.subplots(figsize=(10, 4), facecolor="black")
                sns.boxplot(
                    x="classe_atratividade", y=var, data=df_servicos,
                    hue="classe_atratividade", palette=palette_custom,
                    dodge=False, ax=ax
                )
                if ax.get_legend():
                    ax.legend_.remove()
                ax.set_facecolor("black")
                ax.set_title(
                    f"Distribuição de '{title}' por Classe de Atratividade",
                    color="white"
                )
                ax.tick_params(colors="white")

                ax.yaxis.set_major_locator(MaxNLocator(integer=True))
                if var in ["Hospitais", "Centro Saude"]:
                    from matplotlib.ticker import FuncFormatter
                    ax.yaxis.set_major_formatter(
                        FuncFormatter(lambda x, _: str(int(x)))
                    )
                    ax.set_yticks([0, 1, 2, 3, 4])
                    ax.set_ylim(0, 4.5)

                plt.tight_layout()
                imagens.append(
                    html.Img(src=fig_to_base64(fig),
                             style={"display": "block", "margin": "auto",
                                    "margin-bottom": "20px"})
                )
            return html.Div(imagens, style={'backgroundColor': '#000'})

        elif tab == "tab3":
            rich_html = generate_rich_tables_html()
            return html.Div([
                html.Iframe(
                    srcDoc=rich_html,
                    style={"width": "100%", "height": "800px", "border": "none"}
                )
            ], style={'backgroundColor': '#000', 'padding': '20px'})

    return app

# --- CRIAR E EXECUTAR O SEGUNDO Painel ---
def find_free_port():
    while True:
        port = random.randint(8000, 9000)
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(("localhost", port)) != 0:
                return port

## --- CRIAR E EXECUTAR O SEGUNDO Painel ---
app_2 = create_app()           
port = find_free_port()

if __name__ == "__main__":
    app_2.run(jupyter_mode="inline", debug=False, port=port)

# ---- Mensagem final---
print("\033[92m[INFO] Após análise, pode continuar.\033[0m")

<IPython.core.display.Javascript object>

[INFO] Após análise, pode continuar.


In [7]:
import os
import pandas as pd
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript

# hide-me
display(Javascript('window.cellVisibilityManager.hideCells();'))

# --- Pasta específica para guardar as respostas ---
pasta_respostas = ".../respostas"
os.makedirs(pasta_respostas, exist_ok=True)  

# --- Questão de escolha múltipla 1 ---
questao_1 = widgets.HTML(value="<b>9. Com base na matriz de correlações, qual a relação entre a presença de farmácias, supermercados e CTT?</b>")

pergunta_1 = widgets.RadioButtons(
    options=[
        "a) A presença de um destes serviços está associada a maior presença dos outros serviços.",
        "b) A presença de um destes serviços está associada a menor presença dos outros serviços.",
        "c) A presença de um destes serviços não está associada à presença dos outros serviços."
    ],
    description='Escolha uma opção:',
    layout=widgets.Layout(width='600px', height='auto', margin='10px 0 20px 0'),  
    value=None,  
    style={'description_width': 'initial', 'font_size': '12px'}  
)

# --- Questão de escolha múltipla 2 ---
questao_2 = widgets.HTML(value="<b>10. Com base na matriz de correlações, como se caracteriza a relação entre população com 65 anos ou mais e o conjunto de serviços analisados?</b>")

pergunta_2 = widgets.RadioButtons(
    options=[
        "a) Os edifícios com mais população tendem a ter menor oferta de serviços.",
        "b) Os edifícios com mais população tendem a ter maior oferta de serviços.",
        "c) Os edifícios com mais população não apresentam relação com a oferta de serviços."
    ],
    description='Escolha uma opção:',
    layout=widgets.Layout(width='800px', height='auto', margin='10px 0 20px 0'),  
    value=None,  
    style={'description_width': 'initial', 'font_size': '12px'}  
)  

# --- Botão para gravar a resposta ---
botao_gravar = widgets.Button(description='Gravar resposta', button_style='info')
output = widgets.Output()

# --- Função para gravar a respostas ---
def gravar_resposta(b):
    resposta_1 = pergunta_1.value
    resposta_2 = pergunta_2.value

    if resposta_1 is None or resposta_2 is None:
        with output:
            clear_output()
            print("Por favor, preencha todas as respostas antes de gravar.")
        return

    dados = {
        "Questão 1 (Escolha Múltipla)": [resposta_1],
        "Questão 2 (Escolha Múltipla)": [resposta_2],
        "Data": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
    }

   
    ficheiro = os.path.join(pasta_respostas, "respostas_correlacao.xlsx")
    df = pd.DataFrame(dados)

    # --- Guardar as respostas no ficheiro Excel ---
    if not os.path.isfile(ficheiro):
        with output:
            clear_output()
            print(f"O ficheiro {ficheiro} não existe. A criar o ficheiro...")
        df.to_excel(ficheiro, index=False, engine='openpyxl')  # Specify engine openpyxl
        with output:
            clear_output()
            print(f"Ficheiro criado com sucesso em: {ficheiro}")
    else:
        df_existente = pd.read_excel(ficheiro, engine='openpyxl')  # Specify engine openpyxl
        df_final = pd.concat([df_existente, df], ignore_index=True)
        df_final.to_excel(ficheiro, index=False, engine='openpyxl')
        with output:
            clear_output()
            print(f"Resposta adicionada ao ficheiro {ficheiro}")

    # Limpar as opções seleccionadas e apresentar mensagem de sucesso
    with output:
        clear_output()
        print("Respostas gravadas com sucesso! Obrigado.")
        pergunta_1.value = None
        pergunta_2.value = None

botao_gravar.on_click(gravar_resposta)

# --- Apresentar instruções antes das questões ---
display(widgets.HTML("""
<b style="font-size: 18px;">Por favor, responda às 2 questões:</b><br><br>
"""))

# --- Apresentar as questões, opções e botão ---
display(questao_1, pergunta_1, questao_2, pergunta_2, botao_gravar, output)

<IPython.core.display.Javascript object>

HTML(value='\n<b style="font-size: 18px;">Por favor, responda às 2 questões:</b><br><br>\n')

HTML(value='<b>9. Com base na matriz de correlação, o que indica a relação entre distancia_media_servicos_log …

RadioButtons(description='Escolha uma opção:', layout=Layout(height='auto', margin='10px 0 20px 0', width='600…

HTML(value='<b>10. Com base na matriz de correlação, o que se pode concluir sobre a relação entre a população …

RadioButtons(description='Escolha uma opção:', layout=Layout(height='auto', margin='10px 0 20px 0', width='800…

Button(button_style='info', description='Gravar resposta', style=ButtonStyle())

Output()

<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>

Seguinte: [Conclusões (Insights)](8insights_base.ipynb)

<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>